In [1]:
#
# LSTM모델을 이용해서 주가예측
#
import os
import settings
import pandas as pd        # 라이브러리
import numpy as np         # 라이브러리
from keras.models import Sequential      # 딥러닝을 구동하는 데 필요한 케라스 함수
from keras.layers import Dense           # 딥러닝을 구동하는 데 필요한 케라스 함수
from keras.layers import LSTM            # 딥러닝을 구동하는 데 필요한 케라스 함수
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import math
from sklearn.metrics import mean_squared_error
import pymysql         # 파이썬에서 mysql연동시켜주는 라이브러리

#
# DB테이블 값 조회 (SELECT)
#
connection = pymysql.connect(host='222.122.86.187', port=3306, user='geniuses777', password='stock7840',
                       db='geniuses777', charset='utf8')

# 오차율 가져오기
try:
    with connection.cursor() as cursor:
        sql = "select accuracy from stock_hye WHERE company_name='한화'"
        cursor.execute("set names utf8")
        cursor.execute(sql)
        result_accuracy = cursor.fetchone()
        
        for i in result_accuracy:
            accuracy = i
            
finally:
    connection.close()

    
# 데이터셋 생성 함수
look_back = 1
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i + look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)
 
#
# 저장되어있는 주식데이터 불러오기
#
sydtpath = os.path.join(settings.BASE_DIR, 'chart_data/%s' % (settings.get_today_str()))
stock_code = "hanhwa"
fullpath = sydtpath + os.path.sep + stock_code + '.csv'
pandf = pd.read_csv(fullpath, index_col="Date")

# 데이터 전처리
nparr = pandf['Close'].values[1:]     # 맨처음 'Close'데이터부터 차례대로 nparr에 저장
print(nparr)
nparr.astype('float32')    # float형으로 변환
print(nparr)
nparr = nparr.reshape(-1,1)
print(nparr)
 
# 정규화 (0~1사이의 값으로 바꿔준다)
scaler = MinMaxScaler(feature_range=(0, 1))
nptf = scaler.fit_transform(nparr)
 
# 학습용, 테스트용 데이터로 나누기 (90%를 학습용 데이터, 10%를 테스트용 데이터)
train_size = int(len(nptf) * 0.9)
test_size = len(nptf) - train_size
train, test = nptf[0:train_size], nptf[train_size:len(nptf)]
print(len(train), len(test))
 
# 학습을 위한 데이터셋 생성하기 (학습용, 테스트용으로 구분)
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
 
# RNN모델은 3차원 데이터
# trainX, testX값을 [samples, time steps, features] 형태로 reshape
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
 
# LSTM모델
model = Sequential()                                                # 딥러닝 구조, 층을 설정
model.add(LSTM(20, input_shape=(1, 1)))     # (timestep, feature)   # 층이 추가됨 (add)   # 입력층, 첫번째 은닉층                   
model.add(Dense(1))                                                 # 출력층 (하나)
# 모델을 컴파일 (컴퓨터가 알아들을 수 있도록)   # 오차함수, 최적화 방법
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])   # metrics : 모델 수행 결과를 나타내게 설정 (과적합 문제 방지)
model.fit(trainX, trainY, epochs=1000, batch_size=50, verbose=2)      # 모델을 실제로 수행     # batch_size : 전체 데이터를 10개씩 사용
                                                                                               # verbose(로깅)  2 : epoch당 나오게

# 예측값 평가하기 (얼마나 정확한지)
testPredict = model.predict(testX)
testPredict = scaler.inverse_transform(testPredict)                 # testPredict : 예측 값
testY = scaler.inverse_transform(testY)                             # testY : 실제 값
testScore = math.sqrt(mean_squared_error(testY, testPredict))       # mean_squared_error : 평균 제곱근 오차
print('Train Score: %.2f RMSE' % testScore)                         # 예측 값과 실제 값 차이 출력
 
# 예측 데이터 출력
lastX = nptf[-1]
lastX = np.reshape(lastX, (1, 1, 1))
lastY = model.predict(lastX)
predict = scaler.inverse_transform(lastY)                    # 정규화 시킨 값을 역변환
print('Predict the Close value of final day: %d' % predict)  # 데이터 입력 마지막 다음날 종가 예측
    
# 차트출력, 저장
font_path = "C:/Windows/Fonts/a옛날목욕탕L.ttf"
fontprop = fm.FontProperties(fname=font_path, size=15)

plt.plot(testPredict)
plt.plot(testY)

plt.title('hanhwa predict graph')

#plt.savefig("./chart_picture/hanhwa.png",dpi=300)
plt.savefig("C:\source\SPF\chart_picture\hanhwa.png", dpi=300)
#plt.show()

# 사진 데이터 binary형식으로 바꿔주는 함수
def convertToBinaryData(filename):
    #Convert digital data to binary format
    with open(filename, 'rb') as file:
        binaryData = file.read()
    return binaryData


# 
# DB테이블 값 삽입 (INSERT)
#
# MySQL Connection 연결
connection = pymysql.connect(host='222.122.86.187', port=3306, user='geniuses777', password='stock7840',
                       db='geniuses777', charset='utf8')
try:
    with connection.cursor() as cursor:
        sql = 'INSERT INTO stock_hye (company_name, stock_price, image) VALUES (%s, %s, %s) ON DUPLICATE KEY UPDATE stock_price = VALUES(stock_price), image = VALUES(image)'
        image = convertToBinaryData("C:\source\SPF\chart_picture\hanhwa.png")
        cursor.execute(sql, ('한화', int(predict), image))          # 넣으려는 값
    connection.commit()
    
finally:
    connection.close()

Using TensorFlow backend.


[21888 23201 23833 ... 27450 26750 26450]
[21888 23201 23833 ... 27450 26750 26450]
[[21888]
 [23201]
 [23833]
 ...
 [27450]
 [26750]
 [26450]]
2308 257


C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Epoch 1/1000
 - 1s - loss: 0.1571 - acc: 4.3365e-04
Epoch 2/1000
 - 0s - loss: 0.0614 - acc: 4.3365e-04
Epoch 3/1000
 - 0s - loss: 0.0212 - acc: 8.6730e-04
Epoch 4/1000
 - 0s - loss: 0.0141 - acc: 8.6730e-04
Epoch 5/1000
 - 0s - loss: 0.0126 - acc: 8.6730e-04
Epoch 6/1000
 - 0s - loss: 0.0112 - acc: 8.6730e-04
Epoch 7/1000
 - 0s - loss: 0.0098 - acc: 8.6730e-04
Epoch 8/1000
 - 0s - loss: 0.0085 - acc: 8.6730e-04
Epoch 9/1000
 - 0s - loss: 0.0071 - acc: 8.6730e-04
Epoch 10/1000
 - 0s - loss: 0.0059 - acc: 8.6730e-04
Epoch 11/1000
 - 0s - loss: 0.0048 - acc: 8.6730e-04
Epoch 12/1000
 - 0s - loss: 0.0038 - acc: 8.6730e-04
Epoch 13/1000
 - 0s - loss: 0.0029 - acc: 8.6730e-04
Epoch 14/1000
 - 0s - loss: 0.0023 - acc: 8.6730e-04
Epoch 15/1000
 - 0s - loss: 0.0017 - acc: 8.6730e-04
Epoch 16/1000
 - 0s - loss: 0.0013 - acc: 8.6730e-04
Epoch 17/1000
 - 0s - loss: 0.0010 - acc: 8.6730e-04
Epoch 18/1000
 - 0s - loss: 8.6183e-04 - acc: 8.6730e-04
Epoch 19/1000
 - 0s - loss: 7.4539e-04 - acc: 8.673

Epoch 146/1000
 - 0s - loss: 5.6839e-04 - acc: 8.6730e-04
Epoch 147/1000
 - 0s - loss: 5.7231e-04 - acc: 8.6730e-04
Epoch 148/1000
 - 0s - loss: 5.6999e-04 - acc: 8.6730e-04
Epoch 149/1000
 - 0s - loss: 5.8871e-04 - acc: 8.6730e-04
Epoch 150/1000
 - 0s - loss: 5.7227e-04 - acc: 8.6730e-04
Epoch 151/1000
 - 0s - loss: 5.6875e-04 - acc: 8.6730e-04
Epoch 152/1000
 - 0s - loss: 5.6552e-04 - acc: 8.6730e-04
Epoch 153/1000
 - 0s - loss: 5.7093e-04 - acc: 8.6730e-04
Epoch 154/1000
 - 0s - loss: 5.6677e-04 - acc: 8.6730e-04
Epoch 155/1000
 - 0s - loss: 5.6714e-04 - acc: 8.6730e-04
Epoch 156/1000
 - 0s - loss: 5.6615e-04 - acc: 8.6730e-04
Epoch 157/1000
 - 0s - loss: 5.6991e-04 - acc: 8.6730e-04
Epoch 158/1000
 - 0s - loss: 5.6788e-04 - acc: 8.6730e-04
Epoch 159/1000
 - 0s - loss: 5.6407e-04 - acc: 8.6730e-04
Epoch 160/1000
 - 0s - loss: 5.6805e-04 - acc: 8.6730e-04
Epoch 161/1000
 - 0s - loss: 5.6887e-04 - acc: 8.6730e-04
Epoch 162/1000
 - 0s - loss: 5.6673e-04 - acc: 8.6730e-04
Epoch 163/1000

...

Epoch 429/1000
 - 0s - loss: 5.6902e-04 - acc: 8.6730e-04
Epoch 430/1000
 - 0s - loss: 5.6666e-04 - acc: 8.6730e-04
Epoch 431/1000
 - 0s - loss: 5.6648e-04 - acc: 8.6730e-04
Epoch 432/1000
 - 0s - loss: 5.7634e-04 - acc: 8.6730e-04
Epoch 433/1000
 - 0s - loss: 5.7132e-04 - acc: 8.6730e-04
Epoch 434/1000
 - 0s - loss: 5.6546e-04 - acc: 8.6730e-04
Epoch 435/1000
 - 0s - loss: 5.8360e-04 - acc: 8.6730e-04
Epoch 436/1000
 - 0s - loss: 5.7438e-04 - acc: 8.6730e-04
Epoch 437/1000
 - 0s - loss: 5.6503e-04 - acc: 8.6730e-04
Epoch 438/1000
 - 0s - loss: 5.6472e-04 - acc: 8.6730e-04
Epoch 439/1000
 - 0s - loss: 5.6934e-04 - acc: 8.6730e-04
Epoch 440/1000
 - 0s - loss: 5.6937e-04 - acc: 8.6730e-04
Epoch 441/1000
 - 0s - loss: 5.7280e-04 - acc: 8.6730e-04
Epoch 442/1000
 - 0s - loss: 5.6411e-04 - acc: 8.6730e-04
Epoch 443/1000
 - 0s - loss: 5.6405e-04 - acc: 8.6730e-04
Epoch 444/1000
 - 0s - loss: 5.6538e-04 - acc: 8.6730e-04
Epoch 445/1000
 - 0s - loss: 5.8175e-04 - acc: 8.6730e-04
Epoch 446/1000

 - 0s - loss: 5.6320e-04 - acc: 8.6730e-04
Epoch 571/1000
 - 0s - loss: 5.6447e-04 - acc: 8.6730e-04
Epoch 572/1000
 - 0s - loss: 5.7725e-04 - acc: 8.6730e-04
Epoch 573/1000
 - 0s - loss: 5.7395e-04 - acc: 8.6730e-04
Epoch 574/1000
 - 0s - loss: 5.7952e-04 - acc: 8.6730e-04
Epoch 575/1000
 - 0s - loss: 5.6890e-04 - acc: 8.6730e-04
Epoch 576/1000
 - 0s - loss: 5.6422e-04 - acc: 8.6730e-04
Epoch 577/1000
 - 0s - loss: 5.7227e-04 - acc: 8.6730e-04
Epoch 578/1000
 - 0s - loss: 5.7674e-04 - acc: 8.6730e-04
Epoch 579/1000
 - 0s - loss: 5.6436e-04 - acc: 8.6730e-04
Epoch 580/1000
 - 0s - loss: 5.6904e-04 - acc: 8.6730e-04
Epoch 581/1000
 - 0s - loss: 5.7056e-04 - acc: 8.6730e-04
Epoch 582/1000
 - 0s - loss: 5.7265e-04 - acc: 8.6730e-04
Epoch 583/1000
 - 0s - loss: 5.6852e-04 - acc: 8.6730e-04
Epoch 584/1000
 - 0s - loss: 5.7286e-04 - acc: 8.6730e-04
Epoch 585/1000
 - 0s - loss: 5.7404e-04 - acc: 8.6730e-04
Epoch 586/1000
 - 0s - loss: 5.8496e-04 - acc: 8.6730e-04
Epoch 587/1000
 - 0s - loss: 

Epoch 712/1000
 - 0s - loss: 5.9753e-04 - acc: 8.6730e-04
Epoch 713/1000
 - 0s - loss: 5.6539e-04 - acc: 8.6730e-04
Epoch 714/1000
 - 0s - loss: 5.6622e-04 - acc: 8.6730e-04
Epoch 715/1000
 - 0s - loss: 5.6543e-04 - acc: 8.6730e-04
Epoch 716/1000
 - 0s - loss: 5.6608e-04 - acc: 8.6730e-04
Epoch 717/1000
 - 0s - loss: 5.7667e-04 - acc: 8.6730e-04
Epoch 718/1000
 - 0s - loss: 5.6305e-04 - acc: 8.6730e-04
Epoch 719/1000
 - 0s - loss: 5.6344e-04 - acc: 8.6730e-04
Epoch 720/1000
 - 0s - loss: 5.6935e-04 - acc: 8.6730e-04
Epoch 721/1000
 - 0s - loss: 5.6490e-04 - acc: 8.6730e-04
Epoch 722/1000
 - 0s - loss: 5.7919e-04 - acc: 8.6730e-04
Epoch 723/1000
 - 0s - loss: 5.7080e-04 - acc: 8.6730e-04
Epoch 724/1000
 - 0s - loss: 5.6218e-04 - acc: 8.6730e-04
Epoch 725/1000
 - 0s - loss: 5.6372e-04 - acc: 8.6730e-04
Epoch 726/1000
 - 0s - loss: 5.7015e-04 - acc: 8.6730e-04
Epoch 727/1000
 - 0s - loss: 5.7842e-04 - acc: 8.6730e-04
Epoch 728/1000
 - 0s - loss: 5.6908e-04 - acc: 8.6730e-04
Epoch 729/1000

 - 0s - loss: 5.8702e-04 - acc: 8.6730e-04
Epoch 854/1000
 - 0s - loss: 5.6638e-04 - acc: 8.6730e-04
Epoch 855/1000
 - 0s - loss: 5.6604e-04 - acc: 8.6730e-04
Epoch 856/1000
 - 0s - loss: 5.6568e-04 - acc: 8.6730e-04
Epoch 857/1000
 - 0s - loss: 5.6459e-04 - acc: 8.6730e-04
Epoch 858/1000
 - 0s - loss: 5.6432e-04 - acc: 8.6730e-04
Epoch 859/1000
 - 0s - loss: 5.7448e-04 - acc: 8.6730e-04
Epoch 860/1000
 - 0s - loss: 5.7729e-04 - acc: 8.6730e-04
Epoch 861/1000
 - 0s - loss: 5.6526e-04 - acc: 8.6730e-04
Epoch 862/1000
 - 0s - loss: 5.6597e-04 - acc: 8.6730e-04
Epoch 863/1000
 - 0s - loss: 5.6403e-04 - acc: 8.6730e-04
Epoch 864/1000
 - 0s - loss: 5.6287e-04 - acc: 8.6730e-04
Epoch 865/1000
 - 0s - loss: 5.6558e-04 - acc: 8.6730e-04
Epoch 866/1000
 - 0s - loss: 5.6649e-04 - acc: 8.6730e-04
Epoch 867/1000
 - 0s - loss: 5.6376e-04 - acc: 8.6730e-04
Epoch 868/1000
 - 0s - loss: 5.6653e-04 - acc: 8.6730e-04
Epoch 869/1000
 - 0s - loss: 5.6607e-04 - acc: 8.6730e-04
Epoch 870/1000
 - 0s - loss: 

Epoch 995/1000
 - 0s - loss: 5.6900e-04 - acc: 8.6730e-04
Epoch 996/1000
 - 0s - loss: 5.7889e-04 - acc: 8.6730e-04
Epoch 997/1000
 - 0s - loss: 5.6516e-04 - acc: 8.6730e-04
Epoch 998/1000
 - 0s - loss: 5.6331e-04 - acc: 8.6730e-04
Epoch 999/1000
 - 0s - loss: 5.7425e-04 - acc: 8.6730e-04
Epoch 1000/1000
 - 0s - loss: 5.9623e-04 - acc: 8.6730e-04
Train Score: 689.44 RMSE
Predict the Close value of final day: 26772
